# Linear regression 

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [ ]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('mieszkania.csv', sep=',')
df.describe()

,m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
count,200.00000,200.000000,200.0000,200.000000,200.00000,2.000000e+02
mean,80.08000,1.990000,1.6200,1961.645000,0.49500,5.079195e+05
std,38.46964,0.756812,0.6144,30.383462,0.50123,2.723625e+05
min,15.00000,1.000000,1.0000,1912.000000,0.00000,1.025720e+05
25%,49.00000,1.000000,1.0000,1933.750000,0.00000,3.051080e+05
50%,75.00000,2.000000,2.0000,1961.000000,0.00000,4.556925e+05
75%,111.25000,3.000000,2.0000,1989.000000,1.00000,6.628898e+05
max,149.00000,3.000000,3.0000,2015.000000,1.00000,1.102309e+06


In [4]:
dzielnice = df['dzielnica'].unique()
dzielnica2labels = dict(zip(dzielnice, np.arange(len(dzielnice))))
dzielnica2labels

{'grodziskowo': 2, 'mokotowo': 0, 'ochotowo': 1, 'wolowo': 3}

In [5]:
df['dzielnica'] = df['dzielnica'].apply(lambda x: dzielnica2labels[x])
cena = df.cena.values
N = cena.shape[0]

Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [6]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    before_square = np.log(ys+1) - np.log(ps+1)
    before_mean = before_square**2
    return before_mean.mean()

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [7]:
msle(cena.mean()*np.ones(N), cena)

0.3915253538257009

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [8]:
# computed via LSE

pre_exp = np.sum(np.log(cena+1)) / N
min_msle = np.exp(pre_exp)-1
print(min_msle)
msle(min_msle*np.ones(N), cena)

431435.4915823427


0.3648896122133589

In [9]:
# check
min([msle(i*np.ones(N), cena) for i in range(0, cena.max(), cena.max()//10000)])

0.36488961350271654

Now, let's implement a standard linear regression model. 

In [10]:
def linear_regression(x_tr, y_tr, x_te):
  x_te = np.concatenate([x_te, np.ones(x_te.shape[0])[:,np.newaxis]], axis=1)
  x_tr = np.concatenate([x_tr, np.ones(x_tr.shape[0])[:,np.newaxis]], axis=1)
  w = np.linalg.inv(x_tr.T @ x_tr) @ x_tr.T @ y_tr
  return x_te @ w

y_pred = linear_regression(df.values[:,:-1], cena, df.values[:,:-1])

msle(y_pred, cena)

##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


nan

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [11]:
y_pred = linear_regression(df.values[:,:-1], np.log(cena+1), df.values[:,:-1])

msle(np.exp(y_pred)-1, cena)

0.04755928928241612

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [ ]:
df.corr(method='spearman')

,m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
m2,1.000000,-0.003630,0.935049,0.893359,0.157851,-0.008228,0.913324
dzielnica,-0.003630,1.000000,-0.014360,-0.025890,-0.153419,-0.066835,-0.257462
ilość_sypialni,0.935049,-0.014360,1.000000,0.782058,0.157404,0.039276,0.862540
ilość_łazienek,0.893359,-0.025890,0.782058,1.000000,0.205529,-0.032771,0.814080
rok_budowy,0.157851,-0.153419,0.157404,0.205529,1.000000,-0.021221,0.203255
parking_podziemny,-0.008228,-0.066835,0.039276,-0.032771,-0.021221,1.000000,0.027801
cena,0.913324,-0.257462,0.862540,0.814080,0.203255,0.027801,1.000000


In [ ]:
df = pd.concat([df, (pd.get_dummies(df['dzielnica'])*df['m2']).iloc[:,:4]], axis=1)
df['m2_per_room'] = df['m2']/(df['ilość_sypialni'] + df['ilość_łazienek'])
###############################################
# TODO: Implement the feature engineering part #
###############################################

In [ ]:
test = df.sample(frac=0.2)
train = df.drop(index=test.index)


test = test.values
train = train.values

y_pred1 = linear_regression(train[:,:-1], np.log(train[:,-1]+1), train[:,:-1])
y_pred2 = linear_regression(train[:,:-1], np.log(train[:,-1]+1), test[:,:-1])

print('MSLE on train set', msle(np.exp(y_pred1)-1, train[:,-1]))
print('MSLE on test set', msle(np.exp(y_pred2)-1, test[:,-1]))
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

MSLE on train set 0.050454732840081326
MSLE on test set 0.03765520318177057
